# Computational Complexity and Big-O Notation
#### Jack Bennetto

One of the goals of computer programming is writing code that runs quickly. It's not the most important goal – generally it's much more important to write maintainable, readable code – but in some cases performance is critical. The first step to writing fast code is understanding how to measure it.

## Agenda

* Motivation
* Big-O Notation
* Some Examples and Tests
* Worst, Best, and Average Cases
* Some Math
* Other Asympotic notations, inputs, and stuff to measure

## Motivation

We want to be able describe the efficiency of a particular algorithm.

We can't just talk about how long it takes to run a function. Different computers and languages work differently and run at different speeds. There's really no easy way to talk about how long a program will take without running it, and that will only help us in certain circumstances.

What we can do is estimate how the computer time will grow for larger and larger input.

It turns out this is really important. When writing production code we mostly test code on small inputs, usually smaller than we'd see in the real world. If the code – even a small part of the code is highly dependent on the size of the input, it might run fine initially but fall apart in a crisis. Large software systems undergo *scalability testing*, but it's best to find these problems earlier.

In data science we frequently deal with *big data*. In such cases it's particularly important that the computation time (and memory usage) doesn't grow much faster than the size of the input data.

## Big-O Notation


For now we'll imagine a program with an input of length $n$ and we want to understand how it behaves.

We have a couple important concepts that we want to express mathematically. First, we don't really care about constant factors. So if a program will $n$ seconds to complete, or $2n$, or $100n$, we want to treat those as being all the same, but if it takes $n^2$ seconds, that's something else.

Second, we're interested in limiting behavior, how a system behaves with very large input. If the one part of the program takes $n$ seconds to run, and the other part takes $0.00001n^2$ seconds, eventually, for large enough $n$, the second part will dominate and the whole thing will be proportional to $n^2$. 

The way we express this is called "big-O notation." For example, if a program will take $5 + 5n + 2n^2$ seconds to run, it's said to be $O(n^2)$. If it always takes the same amount of time independent of $n$ it's $O(1)$.

Mathematically, we say that 

$$f(n) = O(g(n)) \text{ as } n \to \infty$$

if and only if there exists numbers $M$ and $n_0$ such that

$$|f(n)| \le M|g(n)| \text{ for all } n \ge n_0$$

I'll talk more later what that means, but the basic points are

 * constants don't matter (note that $M$ term), and
 * only the fastest-growing term matters.
 


## Computational Complexity

So how do we consider that in code? Consider the function:

In [ ]:
import scipy.stats as scs

In [ ]:
def my_sum(lst):
    '''Sum the numbers in a sequence'''
    result = 0
    for x in lst:
        result += x
    return result

Question: How long should this take as a function of the size of the input list?

Let's test it. We're drawing a lot of numbers from a random distribution, so we'll create the distribution object first. Note that `%%timeit` measures the time to run the entire cell, *excluding* the rest of that line.

In [ ]:
dist = scs.norm(0, 1)

In [ ]:
%%timeit lst = dist.rvs(1000)
my_sum(lst)

In [ ]:
%%timeit lst = dist.rvs(10000)
my_sum(lst)

In [ ]:
%%timeit lst = dist.rvs(100000)
my_sum(lst)

In [ ]:
%%timeit lst = dist.rvs(1000000)
my_sum(lst)

So it looks as if the time to run it is propotional to the size of the input, a.k.a., it's an $O(n)$ algoritm. Here's how we'd figure that out.

In [ ]:
def my_sum(lst):
    result = 0         # O(1)
    for x in lst:      # O(n)
        result += x    #   O(1)
    return result      # O(1)

How about this? Note we're iterating over the indices rather than the elements, but that shouldn't make a difference.

In [ ]:
def maxdiff(lst):
    '''find the maximum absolute difference of elements in lst'''
    result = 0
    for i in xrange(len(lst)):
        for j in xrange(len(lst)):
            if abs(lst[i] - lst[j]) > result:
                result = abs(lst[i] - lst[j])
    return result

How long should this take as a function of n?

Let's try it out! We're just going to go up by factors of 2, since this is a lot slower.

In [ ]:
%%timeit lst = dist.rvs(100)
maxdiff(lst)

In [ ]:
%%timeit lst = dist.rvs(200)
maxdiff(lst)

In [ ]:
%%timeit lst = dist.rvs(400)
maxdiff(lst)

In [ ]:
%%timeit lst = dist.rvs(800)
maxdiff(lst)

But wait! Our function is considering every pair of numbers twice, but since we're taking the absolute value of the differences we don't need to.

In [ ]:
def maxdiff(lst):
    '''find the maximum absolute difference of elements in lst'''
    result = 0
    for i in xrange(len(lst)):
        for j in xrange(i):
            if abs(lst[i] - lst[j]) > result:
                result = abs(lst[i] - lst[j])
    return result

In [ ]:
%%timeit lst = dist.rvs(100)
maxdiff(lst)

In [ ]:
%%timeit lst = dist.rvs(200)
maxdiff(lst)

In [ ]:
%%timeit lst = dist.rvs(400)
maxdiff(lst)

In [ ]:
%%timeit lst = dist.rvs(800)
maxdiff(lst)

In the first case we're looking at the square of n:
$$\sum_{i=0}^{n-1} n = n^2$$

In the second case we have
$$\sum_{i=0}^{n-1} i = \frac{n(n-1)}{2} \approx \frac{n^2}{2}$$

To look at it graphically, for the first case for $n=6$ we have

`
. . . . . .
. . . . . .
. . . . . .
. . . . . .
. . . . . .
. . . . . .
`

While for the second case we have

`
.
. .
. . .
. . . .
. . . . .
`

We're only looking at half of the square here, so we save a factor of two, but **the computational complexity is the same**. These sorts of double loops are pretty common.

It's pretty common the obvious solution to a problem is $O(n^2)$ but there's a faster solution. Could we do better here?

### Built-in functions - lookup in a list

In order to figure out the overall computational complexity of our code, we need to know the complexity for built-in functions.

What's the computational complexity of checking if something is in a list?

In [ ]:
%%timeit lst = dist.rvs(1000); x = dist.rvs()
x in lst

In [ ]:
%%timeit lst = dist.rvs(10000); x = dist.rvs()
x in lst

In [ ]:
%%timeit lst = dist.rvs(100000); x = dist.rvs()
x in lst

In [ ]:
%%timeit lst = dist.rvs(1000000); x = dist.rvs()
x in lst

If we wrote it as a function it would look a bit like this.

In [ ]:
def is_in_list(x, lst):
    for element in list:  # O(n)
        if x == element:  # O(1)
            return True
    return False          # O(1)

### Best, Worst, and Average Cases

We assumed above that we have to look through the entire list to find something, but what if the element we're looking for is the first one? That's what's called the **best case**; if that's true here we can solve the problem in $O(1)$ time.

Knowing the best case isn't that useful, because it doesn't happen very often, and it's not safe to rely on it. A better approach is to look at the **average case**, for a typical situation. For the average case above (assuming the element is in the list) we'll have to look at half of the elements before we find it. We generally care about the average case, because we'll be running our algoritm many times and the average performance matters.

We also need to consider the **worst case**, for a couple reasons. First, even though the worst case might be rare, we need still may need to handle it without catastrophic failure. If the worst-case performance will cause our server to crash, we have a problem.

Second, the worst-case scenario, though rare on random data, might be common on real-world situations. Quicksort (on the common sorting algorithms) is $O(\log n)$ complexity in the average case but $O(n^2)$ in the worst case. That might seem ok, except the worst case occurs when the list is already sorted – a fairly common situation. To avoid this we randomize the list before sorting.

### Lookup in a sorted list
Often we can perform operations faster if the data is organized in a particular way to start with. Suppose we knew that the list was sorted in increasing order. How could we look for something in the list without checking every single element?

Let's write a function to do this. 

In [ ]:
def is_in_sorted_list(x, lst):
    lower = 0
    upper = len(lst) - 1
    while lower - 1  < upper:
        current = (lower + upper) // 2
        if x == lst[current]:
            return True
        elif x < lst[current]:
            upper = current - 1
        else:
            lower = current + 1
    return False

In [ ]:
%%timeit lst = sorted(dist.rvs(100)); x = dist.rvs()
is_in_sorted_list(x, lst)

In [ ]:
%%timeit lst = sorted(dist.rvs(1000)); x = dist.rvs()
is_in_sorted_list(x, lst)

In [ ]:
%%timeit lst = sorted(dist.rvs(10000)); x = dist.rvs()
is_in_sorted_list(x, lst)

In [ ]:
%%timeit lst = sorted(dist.rvs(100000)); x = dist.rvs()
is_in_sorted_list(x, lst)

In [ ]:
%%timeit lst = sorted(dist.rvs(1000000)); x = dist.rvs()
is_in_sorted_list(x, lst)

It looks as if it's taking longer to for the larger lists, maybe growing linearly as the lists grow geometrically. That's a $O(\log n)$ growth. In general, log terms in computational complexity occur when you repeatedly divide the data.

## Lookup in a set
What about a set?

In [ ]:
%%timeit s = set(dist.rvs(1000)); x = dist.rvs()
x in s

In [ ]:
%%timeit s = set(dist.rvs(10000)); x = dist.rvs()
x in s

In [ ]:
%%timeit s = set(dist.rvs(100000)); x = dist.rvs()
x in s

Set lookup (and dictionary lookup) is done using a hash, which I won't talk about now, but the important point is that it's independent of the size of the set, i.e., it's $O(1)$.

### Some Big-O examples

The complexity of some of the built-in functions are in https://wiki.python.org/moin/TimeComplexity

Computational Complexity | Algorithm
----------------|-------------------
$O(1)$          | Hash lookup, append to list
$O(\log n)$     | Tree lookup
$O(n)$          | List lookup
$O(n \log n)$   | List sorting

## The Math

As mentioned above,

$$f(n) = O(g(n)) \text{ as } n \to \infty$$

if and only if there exists numbers $M$ and $n_0$ such that

$$|f(n)| \le M|g(n)| \text{ for all } n \ge n_0$$

To make sense of that let's consider an example. We'll try proving that

$$2n^2 + 6n + 1 \text{ is } O(n^2)$$

To do that, we need to find some numbers $M$ and $n_0$. They can be *any* number, but we have to be able to choose some number that will work for any $n$.

For really really big n, $2n^2 + 6n + 1$ wil be about 2 times $O(n^2)$. Since $M$ can be any number we'll choose something a bit bigger than that so we have a bit of room, maybe $M = 3$.

What about $n_0$? How big does $n$ have to be so that $|2n^2 + 6n + 1| \le 3 \cdot |n^2|$?

We could solve for that. For positive $n$, this becomes

$2n^2 + 6n + 1 \le 3n^2$

or

$n^2 - 6n - 1 \ge 0$

The equality holds at $n = 3$ or $n = -2$; for anything greater than $n$ the inequality holds.

#### A counter example

Can we show that $n^2$ is not $O(n)$? For that we assume that $M$ and $n_0$ exist and try to find a contradition.

So for all $n \ge n_0$, $|n^2| \le M|n|$. If we just consider $n > 0$ that simplifies to

$$n^2 \le Mn$$
$$n \le M$$

If $n > M$ then that will fail, so that isn't true in all values of $n > n_0$ and so $n^2$ is not $O(n)$.

### Other Asymptotic Notation

Note that Big-O notation is just an upper bound, so it's true that
$$5x^2 + 2x + 1 \text{ is } O(x^2)$$
but also that
$$5x^2 + 2x + 1 \text{ is } O(x^3)$$
and
$$5x^2 + 2x + 1 \text{ is } O(e^x)$$

If you want to talk about the lower bound, use the $\Omega$ instead, so

$$5x^2 + 2x + 1 \text{ is } \Omega(x^2)$$
and also
$$5x^2 + 2x + 1 \text{ is } \Omega(x)$$

If you want to get the exact bound, both upper and lower, you use $\Theta$, so

$$5x^2 + 2x + 1 \text{ is } \Theta(x^2)$$

Big-O is more common than Big-$\Theta$ because it's easier to prove; it can be hard to prove a lower bound. Usually when people talk about Big-O they really mean Big-$\Theta$, but I wouldn't recommend being pedantic in an interview.

There's also a little-o and little-$\omega$ that mean that that one function grows much faster or slower than another.


### Dependencies on other variables

Sometimes a computational complexity is a function of multiple variables, not just the size of the input. For example, if multiplying a $n$ x $m$ matrix by an $m$ by $p$ matrix with the trivial approach will be $O(n m p)$.

With multiple inputs we may need to keep multiple terms. If we insert a $k$ elements at the beginning of a list of $n$ elements long, the complexity may depend on both $k$ and $n$ ($O(n + k)$ if it's not stored as a linked list).

### Memory complexity

Usually we care about how long it takes our program to run, but we can also extend this approach to other resources, most commonly the amount of memory needed to run the  program. If the data already is in memory, that's beyond what's aready there, so a function to sort a list that creates a new list will be (at least) $O(n)$, while one that sorts the list in places might only be $O(1)$.

In many big-data situations we may be reading the data from disk and not have the memory to store it in memory. In this case we need an algorithm that's $O(1)$ in memory usage.